# Домашнее задание 3

Выполнил Козин Роман

# Часть 1. Классификатор 128×128

Для ускорения процесса обучения используем не весь датасет ImageNet, возьмём часть классов. Пусть будет 40 классов. Выберем их случайным образом

In [1]:
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import numpy as np
import random
import torch

In [2]:
random.seed(52)
torch.manual_seed(52)
np.random.seed(52)

In [3]:
CLASS_NUM = 40
classes_path = './data/tiny-imagenet-200/tiny-imagenet-200/wnids.txt'
with open(classes_path) as f:
    ids = f.read().split('\n')
selected_classes = random.sample(ids[:-1], CLASS_NUM)
print(selected_classes)

['n01882714', 'n02279972', 'n02699494', 'n02123394', 'n03026506', 'n01910747', 'n04532106', 'n07920052', 'n02058221', 'n07695742', 'n02268443', 'n07753592', 'n03160309', 'n04356056', 'n01784675', 'n02124075', 'n07871810', 'n04417672', 'n02927161', 'n03837869', 'n04540053', 'n09193705', 'n02364673', 'n02909870', 'n01774750', 'n12267677', 'n03404251', 'n01698640', 'n02988304', 'n01770393', 'n03930313', 'n02977058', 'n02963159', 'n01644900', 'n02415577', 'n03992509', 'n07875152', 'n02814533', 'n02395406', 'n02814860']
